# Зайцев Н. ПИ20-1В

<h1 align= 'center'> Работа с полным набором данных IMDb классификация настроений классификаторами: Naive Bayes, Logistic Regression </h1> 

## 1. Классификатор наивный Баес 

### 1A. Загрузим данные

In [1]:
from fastai.text.all import *

In [2]:
path = untar_data(URLs.IMDB)
path.ls()

(#7) [Path('/Users/nikitazaytsev/.fastai/data/imdb/test'),Path('/Users/nikitazaytsev/.fastai/data/imdb/tmp_clas'),Path('/Users/nikitazaytsev/.fastai/data/imdb/imdb.vocab'),Path('/Users/nikitazaytsev/.fastai/data/imdb/unsup'),Path('/Users/nikitazaytsev/.fastai/data/imdb/README'),Path('/Users/nikitazaytsev/.fastai/data/imdb/tmp_lm'),Path('/Users/nikitazaytsev/.fastai/data/imdb/train')]

In [3]:
(path/'train').ls()

(#4) [Path('/Users/nikitazaytsev/.fastai/data/imdb/train/neg'),Path('/Users/nikitazaytsev/.fastai/data/imdb/train/pos'),Path('/Users/nikitazaytsev/.fastai/data/imdb/train/unsupBow.feat'),Path('/Users/nikitazaytsev/.fastai/data/imdb/train/labeledBow.feat')]

Создадим объект загрузчика данных в модель с разбиением выборки на обучающую и проверочную.

In [5]:
device = torch.device('cpu')
reviews_full = TextDataLoaders.from_folder(path, valid_pct=0.15, device=device)

### 1B. Счетчики терминов документов для набора обучающих и валидационных данных. 

Для создания счетчиков важно понимать, что у загрузчика `reviews_full` в аттрибутах `train_ds` и `valid_ds` находятся упорядоченные наборы данных не разбитые на пакеты, которые представляют собой список кортежей, первым элементом которым является нумированный текст в виде списка номеров токенов в словаре.

In [6]:
TrainCounter_full=lambda rev_ind: Counter(reviews_full.train_ds[rev_ind][0].numpy())
ValidCounter_full=lambda rev_ind: Counter(reviews_full.valid_ds[rev_ind][0].numpy())

### 1C. Матрицы терминов-документов

In [7]:
# строим матрицу терминов документа в формате CSR
# т.е. определяем (values, column_indices, row_pointer)

def get_doc_term_matrix(token_counter, n_docs, n_terms):
    
    # ввод:
    #    Объект, TokenCounter
    #    n_docs, количество отзывов в обучающей выборке
    #    n_terms, количество токенов в словаре
    
    # вывод: 
    #    формат CSR разреженное представление матрицы терминов документа в виде
    #    объект scipy.sparse.csr.csr_matrix
    
    # Инициализация массивов
    values = []
    column_indices = []
    row_pointer = []
    row_pointer.append(0)

    for doc_index in range(n_docs):
        column_indices.extend(token_counter(doc_index).keys())
        values.extend(token_counter(doc_index).values())
        # Прикрепите N (количество ненулевых элементов в матрице) к концу массива row_pointer 
        row_pointer.append(len(values))
        
    return scipy.sparse.csr_matrix((values, column_indices, row_pointer),
                                   shape=(len(row_pointer) - 1, n_terms),
                                   dtype=int)

Определим количество документов в обоих выборках

In [8]:
train_ndocs=len(reviews_full.train_ds)
valid_ndocs=len(reviews_full.valid_ds)
train_ndocs, valid_ndocs

(85002, 15000)

Определим количество терминов

In [9]:
n_terms = len(reviews_full.vocab[0])
n_terms

60008

Создадим матрицы для обучающего и валидационного наборов

In [10]:
train_doc_term_full = get_doc_term_matrix(TrainCounter_full, train_ndocs, n_terms)

In [11]:
valid_doc_term_full = get_doc_term_matrix(ValidCounter_full, valid_ndocs, n_terms)

### 1D. Сохранение данных.

Данные сохраним на диск, поскольку объем их слишком большой и может переполнить оперативную память. При хранении подобных данных всегда убеждайтесь, что они включены в ваш файл `gitignore`.

In [12]:
scipy.sparse.save_npz("train_doc_term.npz", train_doc_term_full)

In [13]:
scipy.sparse.save_npz("valid_doc_term.npz", valid_doc_term_full)

In [14]:
with open('reviews_full.pickle', 'wb') as handle:
    pickle.dump(reviews_full, handle, protocol=pickle.HIGHEST_PROTOCOL)

В будущем их можно загрузить, используя следующие команды:

In [15]:
train_doc_term = scipy.sparse.load_npz("train_doc_term.npz")
valid_doc_term = scipy.sparse.load_npz("valid_doc_term.npz")

In [16]:
with open('reviews_full.pickle', 'rb') as handle:
    pickle.load(handle)

### 1E. Обучение классификатора наивный Баес

Создадим более простые литералы для обучения

In [17]:
x = train_doc_term_full
y = np.array([i[1].item() for i in reviews_full.train_ds])

x_valid = valid_doc_term
y_valid = np.array([i[1].item() for i in reviews_full.valid_ds])

Посчитаем количества упоминаний в положительных и отрицательных отзывах каждого токена

In [18]:
C0 = np.squeeze(np.asarray(x[y==0].sum(0)))
C1 = np.squeeze(np.asarray(x[y==1].sum(0)))

In [19]:
C0

array([30686,     0, 21205, ...,     0,     0,     0])

In [20]:
C1

array([37580,     0, 21293, ...,     0,     0,     0])

Определим логарифмы отношений вероятностей для каждого токена:

In [21]:
L1 = (C1+1) / ((y==1).sum() + 1)
L0 = (C0+1) / ((y==0).sum() + 1)

In [22]:
R = np.log(L1/L0)

Рассчитаем смещение модели `b`:

In [23]:
b = np.log((y==1).mean() / (y==0).mean())
print(f'Смещение модели: {b}')

Смещение модели: 0.004141377277707494


Предсказание модели на обучающей выборке:

In [24]:
W = x.sign()
preds = (W @ R + b) > 0
train_accuracy = (preds == y).mean()
print(f'Точность равна {train_accuracy} на обучающей выборке полного набора данных IMDb')

Точность равна 0.44407190419048964 на обучающей выборке полного набора данных IMDb


### Задание 1.
Получить точность предсказаний на проверочной выборке

In [25]:
W = x_valid.sign()
preds_valid = (W @ R + b) > 0
valid_accuracy = (preds_valid == y_valid).mean()
print(f'Точность равна {valid_accuracy} на проверочной выборке полного набора данных IMDb')

Точность равна 0.42606666666666665 на проверочной выборке полного набора данных IMDb


## 2. Классификатор - логистический регрессор (`LogisticRegression`)

С помощью библиотеки scikit-learn можно обучить модель логистической регрессии, в которой объектами являются униграммы. Здесь $C$ - параметр регуляризации.

Используем матрицу терминов-документов как набор признаков: 

In [26]:
from sklearn.linear_model import LogisticRegression

In [27]:
m = LogisticRegression(C=0.1, dual=False,solver = 'liblinear')
# 'liblinear' и 'newton-cg' солверы сходятся
# 'sag', 'saga', и 'lbfgs' расходятся
m.fit(x[:25000], y[:25000])

LogisticRegression(C=0.1, solver='liblinear')

In [28]:
preds = m.predict(x[:25000])
train_accuracy = (preds==y[:25000]).mean()
print(f'Точность модели на обучающей выборке: {train_accuracy}')

Точность модели на обучающей выборке: 0.90228


### Задание 2.
Сформируйте предсказание на валидационной выборке, есть ли переобучение? Как вы думаете почему?

In [29]:
preds_valid = m.predict(x_valid[:25000])
valid_accuracy = (preds_valid==y_valid[:25000]).mean()
print(f'Точность модели на валидационной выборке: {valid_accuracy}')

Точность модели на валидационной выборке: 0.5686
